In [0]:
import torch
#If you enable GPU here, device will be cuda, otherwise it will be cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
#DOWNLOADING DATA SET

# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1ORrHW9moXLcWwg8WY9o-Ulq8X9BAiD1P'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.pkl')  

id = '1eb4gtE8XlN3TcZqzwS18Ik-H7MFAeW4z'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('label.pkl')  

import pickle
input_embeddings = pickle.load(open("train.pkl","rb"))
label = pickle.load(open("label.pkl","rb"))

In [0]:
#SPLIT THE DATASET

# Split into training and testing dataset using scikit-learn

from sklearn.model_selection import train_test_split
train_embeddings, test_embeddings, train_label, test_label = train_test_split(input_embeddings,label,test_size = 0.2, random_state=0)

In [0]:
#GENERATE BATCH

def generate_batch(input_embeddings, label, batch_size):
    idx = np.random.randint(input_embeddings.shape[0], size=batch_size)
    return input_embeddings[idx,:,:],label[idx]

In [0]:
#MODEL

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(n_input, n_hidden, batch_first =True)
        self.linear = nn.Linear(n_hidden, n_class)

    def forward(self, x):        
        
        #forward propogation

        # lstm layer
        x,_ = self.lstm(x)
        # linear layer
        x = self.linear(x[:,-1,:])
        # softmax layer
        x = F.log_softmax(x, dim=1)
              
        return x

In [0]:
import numpy as np
import torch.optim as optim

# Please assign values to these variables by using other variables (instead of hard code)
seq_length = len(train_embeddings[0])
n_input = len(train_embeddings[0][0])
n_class = len(list(set(train_label)))

#Please decide the hyperparameters here by yourself
n_hidden = 128
batch_size = 128
total_epoch = 1000
learning_rate = 0.01
shown_interval = 100

In [7]:
from sklearn.metrics import accuracy_score

net = Net().to(device)
criterion = nn.NLLLoss()

# optimizer to provide higher F1
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(total_epoch):

    input_batch, target_batch = generate_batch(train_embeddings,train_label, batch_size)
    input_batch_torch = torch.from_numpy(input_batch).float().to(device)
    target_batch_torch = torch.from_numpy(target_batch).view(-1).to(device)

    net.train()
    outputs = net(input_batch_torch) 
    loss = criterion(outputs, target_batch_torch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if epoch % shown_interval == shown_interval-1:
        net.eval()
        outputs = net(input_batch_torch) 
        train_loss = criterion(outputs, target_batch_torch)
        _, predicted = torch.max(outputs, 1)
        train_acc= accuracy_score(predicted.cpu().numpy(),target_batch_torch.cpu().numpy())

        print('Epoch: %d, train loss: %.5f, train_acc: %.4f'%(epoch + 1, train_loss.item(), train_acc))

print('Finished Training')

## Prediction
net.eval()
outputs = net(torch.from_numpy(test_embeddings).float().to(device)) 
_, predicted = torch.max(outputs, 1)

from sklearn.metrics import classification_report
print(classification_report(test_label, predicted.cpu().numpy(),digits=4))

Epoch: 100, train loss: 1.42527, train_acc: 0.3281
Epoch: 200, train loss: 1.29339, train_acc: 0.3516
Epoch: 300, train loss: 1.01881, train_acc: 0.5312
Epoch: 400, train loss: 0.91895, train_acc: 0.5234
Epoch: 500, train loss: 1.12914, train_acc: 0.4219
Epoch: 600, train loss: 0.35257, train_acc: 0.7734
Epoch: 700, train loss: 0.23266, train_acc: 0.8125
Epoch: 800, train loss: 0.30713, train_acc: 0.8281
Epoch: 900, train loss: 0.27370, train_acc: 0.8594
Epoch: 1000, train loss: 0.11848, train_acc: 0.9609
Finished Training
              precision    recall  f1-score   support

           0     0.6147    0.7204    0.6634        93
           1     0.9500    0.9344    0.9421       122
           2     0.8947    0.9358    0.9148       109
           3     0.7339    0.6250    0.6751       128

    accuracy                         0.8031       452
   macro avg     0.7983    0.8039    0.7989       452
weighted avg     0.8065    0.8031    0.8026       452

